In [208]:
import os
import triangle as tr
import numpy as np
import plotly.figure_factory as ff
from scipy.io import savemat
from src.scattering_algorithm.scattering_algorithm import *

In [209]:
def traitement_de_(filename):
    base_name = os.path.splitext(os.path.basename(filename))[0]
    print(f"Traitement de l'antenne {base_name}")

In [210]:
point_0 = [-0.099, -0.1]
point_1 = [-0.006, -0.007]
point_2 = [-0.006, 0.007]
point_3 = [-0.099, 0.1]
point_4 = [0.099, 0.1]
point_5 = [0.006, 0.007]
point_6 = [0.006, -0.007]
point_7 = [0.099, -0.1]
point_8 = [-0.066, 0]
point_9 = [0.066, 0]

In [211]:
vertices = np.array([
    point_0, point_1, point_2, point_3, point_4, point_5, point_6, point_7, point_8, point_9
])

segments = np.array([
    [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 0], [8, 9], [2, 5], [1, 6]
])

In [212]:
adaptative_bowtie = {
    'vertices': vertices,
    'segments': segments
}

In [213]:
# Maillage avec contraintes de qualité
mesh_adaptative_bowtie = tr.triangulate(adaptative_bowtie, 'pq30a0.0001Dj')

# Extraire les sommets et les triangles
points = np.zeros((3, mesh_adaptative_bowtie['vertices'].shape[0]))
points[:2, :] = np.array(mesh_adaptative_bowtie['vertices']).T
triangles = np.ones((4, mesh_adaptative_bowtie['triangles'].shape[0]), dtype=int)
triangles[:3, :] = np.array(mesh_adaptative_bowtie['triangles']).T

In [214]:
print(points)
print(f"taille de points : {points.shape}")

[[-9.90000000e-02 -6.00000000e-03 -6.00000000e-03 -9.90000000e-02
   9.90000000e-02  6.00000000e-03  6.00000000e-03  9.90000000e-02
  -6.00000000e-03  6.00000000e-03 -2.60000000e-02  2.60000000e-02
  -5.48058262e-02  5.48058262e-02 -3.04029131e-02  5.48058262e-02
   3.65000000e-02  3.04029131e-02 -5.48058262e-02 -3.65000000e-02
   3.04029131e-02 -3.04029131e-02  0.00000000e+00  0.00000000e+00
  -8.67361738e-19  8.67361738e-19  1.82014565e-02 -1.82014565e-02
  -1.82014565e-02  1.82014565e-02  8.49986521e-03  1.72825969e-02
  -8.49986521e-03 -1.41492193e-02 -1.63716451e-02 -8.49986521e-03
  -1.72825969e-02  8.49986521e-03  1.41492193e-02  1.63716451e-02
   2.54716397e-02 -2.54716397e-02  2.45924058e-02  4.26043696e-02
   4.64142172e-03  3.32999622e-02  4.87050979e-02  4.09405538e-02
   1.32649681e-02  2.64979367e-03 -2.45924058e-02 -4.26043696e-02
  -4.64142172e-03 -3.32999622e-02 -4.87050979e-02 -4.09405538e-02
  -1.32649681e-02 -2.64979367e-03  4.05126863e-02 -4.05126863e-02
  -4.26043

In [215]:
print(triangles)
print(f"taille de triangles : {triangles.shape}")

[[ 48   6  70 ... 178 166 167]
 [ 31   1  68 ... 198 199 197]
 [ 42  25  69 ... 195 162 199]
 [  1   1   1 ...   1   1   1]]
taille de triangles : (4, 324)


In [216]:
# Normalisation pour l'aspect ratio
x_, y_, z_ = points
fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
x_scale = (max(x_) - min(x_)) / fig_scale
y_scale = (max(y_) - min(y_)) / fig_scale
z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

# Création de la figure 3D avec Plotly
fig = ff.create_trisurf(
    x=x_,
    y=y_,
    z=z_,
    simplices=triangles[:3, :].T,
    color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
    show_colorbar=True,
    title="Maillage triangulaire",
    aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
)
# Afficher la figure
fig.show()

In [217]:
# Crée un dictionnaire contenant toutes les données à sauvegarder
data = {
    'p' : points,
    't' : triangles
}
save_folder_name = 'data/antennas_mesh/'
save_file_name = 'adaptative_bowtie.mat'
full_save_path = os.path.join(save_folder_name, save_file_name)  # Chemin complet pour la sauvegarde

# Vérifie si le dossier existe, sinon crée le dossier
if not os.path.exists(save_folder_name): # Vérification et création du dossier si nécessaire
    os.makedirs(save_folder_name)
    print(f"Directory '{save_folder_name}' created.")

# Sauvegarde les données dans le fichier MAT
savemat(full_save_path, data)
print(f"Data saved successfully to {full_save_path}")

Data saved successfully to data/antennas_mesh/adaptative_bowtie.mat


In [218]:
filename_adaptative_mesh_bowtie = 'data/antennas_mesh/adaptative_bowtie.mat'

In [219]:
wave_incident_direction = np.array([0, 0, -1])

In [220]:
start_time = time.time()

traitement_de_(filename_adaptative_mesh_bowtie)
scattering_algorithm(filename_adaptative_mesh_bowtie, 75e7, wave_incident_direction, polarization=np.array([0, 1, 0]), load_from_matlab=False)

elapsed_time = time.time() - start_time
print(f"Temps écoulé pour le traitement de l'antenne étudiée : {elapsed_time:.6f} secondes")
print('\n')

Traitement de l'antenne adaptative_bowtie
length of antenna adaptative_bowtie = 0.198 meter
width of antenna adaptative_bowtie = 0.2 meter
height of antenna adaptative_bowtie = 0.0 meter
Aucune jonction complexe trouvée....
Data saved successfully to data/antennas_mesh1/adaptative_bowtie_mesh1.mat
Data saved successfully to data/antennas_mesh2/adaptative_bowtie_mesh2.mat
Temps écoulé pour le calcul de la matrice Z : 0.280293 secondes
Data saved successfully to data/antennas_impedance/adaptative_bowtie_impedance.mat
Data loaded from data/antennas_mesh2/adaptative_bowtie_mesh2.mat
Data loaded from data/antennas_impedance/adaptative_bowtie_impedance.mat
Temps écoulé pour le calcul du courant : 0.010270 secondes
Data saved successfully to data/antennas_current/adaptative_bowtie_current.mat
Sauvegarde du fichier : adaptative_bowtie_current.mat effectué avec succès !
Fréquence de l'onde incidente : 750000000.0 Hz
Max Current = 0.07459081578666286 [A/m]
adaptative_bowtie antenna surface curre

Temps écoulé pour le traitement de l'antenne étudiée : 2.158437 secondes




In [221]:
filename_adaptative_mesh_bowtie_current = 'data/antennas_current/adaptative_bowtie_current.mat'

In [222]:
*_, current = DataManager_rwg4.load_data(filename_adaptative_mesh_bowtie_current)

In [223]:
current_norm = np.abs(current)
print(current_norm.shape)
average_current_norm = np.mean(current_norm)
print(f"Le courant moyen sur la surface de l'antenne est : {average_current_norm} A")

(449,)
Le courant moyen sur la surface de l'antenne est : 0.004891138909942042 A


In [224]:
# Fonction pour estimer l'erreur basée sur les courants de surface
def estimate_error(triangles, current_norm, average_current_norm):
    errors = np.zeros(len(triangles.T))
    for i in range(len(triangles.T)):
        # Calculer l'erreur pour les courants supérieurs au courant moyen
        current_value = current_norm[i]
        if current_value > average_current_norm:
            errors[i] = current_value - average_current_norm
    return errors

In [225]:
errors = estimate_error(triangles, current_norm, average_current_norm)
print(errors.shape)
print(errors)

(324,)
[0.00000000e+00 0.00000000e+00 1.05918164e-03 6.95095868e-02
 3.39641942e-02 3.36412304e-02 0.00000000e+00 9.71635896e-03
 1.06895188e-02 0.00000000e+00 9.71979714e-03 1.06916349e-02
 2.29740812e-03 2.82555060e-02 1.92729620e-02 1.34223282e-02
 1.19888614e-02 5.94862738e-02 6.97300741e-02 5.94859947e-02
 1.60576693e-03 2.96670139e-03 0.00000000e+00 1.34158056e-02
 1.19867597e-02 0.00000000e+00 1.05372257e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.61121767e-03 2.97240453e-03
 6.95095449e-02 3.36037979e-02 3.40016151e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.31538665e-03 2.82686252e-02
 1.92665826e-02 3.43490883e-03 4.00272345e-03 0.00000000e+00
 3.24418036e-03 1.76633011e-02 3.65278855e-03 1.76568059e-02
 9.24059819e-05 3.35397821e-03 3.38763724e-03 0.00000000e+00
 3.59615074e-03 1.05808289e-04 3.33624974e-03 2.79217348e-02
 1.50933274e-02 3.10013121e-03 4.53184418e-03 2.79328052e-02
 1.50876939e-02 3.14944130e-03 4.55132592e-03 1.74567260e-02
 1.74618997e-02 0

In [226]:
threshold = 1e-03

In [227]:
def refine_mesh(points, triangles, errors, threshold):
    refine_indices = np.where(errors > threshold)[0]
    for idx in refine_indices:
        triangle = triangles[:3, idx]    
        midpoints = (points[:, triangle] + points[:, np.roll(triangle, -1)]) / 2
        points = np.hstack([points, midpoints])
    return points[:2, :].T

In [232]:
# mesh_adaptative_bowtie['vertices'] = refine_mesh(triangles, errors, threshold)
mesh_adaptative_bowtie['vertices'] = refine_mesh(points, triangles, errors, threshold)
mesh_adaptative_bowtie_2 = tr.triangulate(mesh_adaptative_bowtie, 'pq30a0.00001Dj')

In [233]:
# Extraire les sommets et les triangles
points = np.zeros((3, mesh_adaptative_bowtie_2['vertices'].shape[0]))
points[:2, :] = np.array(mesh_adaptative_bowtie_2['vertices']).T
triangles = np.ones((4, mesh_adaptative_bowtie_2['triangles'].shape[0]), dtype=int)
triangles[:3, :] = np.array(mesh_adaptative_bowtie_2['triangles']).T


# Normalisation pour l'aspect ratio
x_, y_, z_ = points
fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
x_scale = (max(x_) - min(x_)) / fig_scale
y_scale = (max(y_) - min(y_)) / fig_scale
z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

# Création de la figure 3D avec Plotly
fig = ff.create_trisurf(
    x=x_,
    y=y_,
    z=z_,
    simplices=triangles[:3, :].T,
    color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
    show_colorbar=True,
    title="Maillage triangulaire",
    aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
)
# Afficher la figure
fig.show()